# PVML SDK Comprehensive Example

This notebook demonstrates a complete workflow using the PVML SDK, showcasing:

1. **Environment Setup** - Loading configuration from .env.local
2. **Client Initialization** - Setting up the PVML client
3. **Workspace Management** - Working with workspaces and users
4. **Data Source Integration** - Connecting to databases
5. **AI Agent Creation** - Building and configuring AI agents
6. **MCP Integration** - Working with Model Context Protocol
7. **Interactive Queries** - Running queries through agents
8. **Session Management** - Managing conversation sessions
9. **Monitoring & Audit** - Tracking usage and performance

## Prerequisites

Before running this notebook, ensure you have:
- PVML SDK installed (`pip install -r requirements.txt`)
- A `.env.local` file with your PVML credentials
- Access to a PVML workspace
- Optional: A database for data source examples


In [ ]:
# Environment Setup and Configuration
import os
from datetime import datetime
from pathlib import Path

from dotenv import load_dotenv


# Load environment variables from .env.local
def load_env_variables(env_file: str = ".env.stage.local") -> bool:
    """Load environment variables from .env.local file using python-dotenv"""
    env_path = Path(env_file)

    if not env_path.exists():
        print(f"Warning: {env_file} not found. Please create it with your PVML credentials.")
        print("\nRequired format:")
        print("PVML_API_KEY=your_api_key_here")
        print("PVML_API_URL=your_api_url_here")
        print("GPT_KEY=your_gpt_key_here")
        print("PVML_WORKSPACE_ID=your_workspace_id_here")
        return False

    # Load environment variables from .env.local
    load_dotenv(env_path)
    print(f"Environment variables loaded from {env_file}")
    return True


# Load environment variables
env_loaded = load_env_variables()

# Validate required environment variables
required_vars = ["PVML_API_KEY", "PVML_API_URL", "PVML_WORKSPACE_ID", "PVML_VIEW_ID", "PVML_AGENT_ID"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    raise ValueError(f"Missing required environment variables: {missing_vars}")

print("All required environment variables loaded successfully")
print(f"API URL: {os.getenv('PVML_API_URL')}")
print(f"Workspace ID: {os.getenv('PVML_WORKSPACE_ID')}")
print(f"View ID: {os.getenv('PVML_VIEW_ID')}")
print(f"Agent ID: {os.getenv('PVML_Agent_ID')}")

print(f"\nConfiguration loaded at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
# Import PVML SDK and Initialize Client
from pvml import Client, Workspace
import utils


# Initialize PVML client
def initialize_pvml_client() -> Client:
    """Initialize and return PVML client"""
    api_key = os.environ.get("PVML_API_KEY")
    if not api_key:
        raise ValueError("PVML_API_KEY not found in environment variables")

    client = Client(api_key=api_key)
    print("PVML client initialized successfully")
    return client


# Initialize client
pvml_client = initialize_pvml_client()
print("Client ready for use")


In [ ]:
def get_workspace(workspace_id: str) -> Workspace:
    """Get the target workspace"""
    if not workspace_id:
        print("PVML_WORKSPACE_ID not found in environment variables")
        print("Available workspaces:")
        workspaces = pvml_client.get_workspaces()
        for ws_id, workspace in workspaces.items():
            print(f"  ID: {ws_id}, Name: {workspace.name}")
        raise ValueError("PVML_WORKSPACE_ID not found in environment variables")

    # Get specific workspace
    workspace = pvml_client.get_workspace(workspace_id)
    print(f"Connected to workspace: {workspace.name}")
    print(f"Workspace ID: {workspace.id}")
    print(f"Description: {workspace.description}")

    return workspace


# Connect to workspace
workspace = get_workspace(os.environ.get("PVML_WORKSPACE_ID"))

# Get current user info
current_user = workspace.get_current_user()
print(f"\nCurrent user: {current_user.email}")
print(f"User role: {current_user.role}")
print(f"User type: {current_user.user_type}")


In [ ]:
ds_view = workspace.get_view(os.environ.get("PVML_VIEW_ID"))
ds_view

In [ ]:
df = await utils.get_pandas_from_execute(ds_view.execute("SELECT * FROM demo_catalog.schema_1.accounts"))
df.head()

In [ ]:
ds_mcp = utils.get_view_mcp(workspace, ds_view)

In [ ]:
ds_agent = workspace.get_agent(os.environ.get("PVML_AGENT_ID"))
ds_agent

In [ ]:
# add the view mcp to the ds agent
ds_agent.update(name=ds_agent.name,
                prompt=ds_agent.prompt,
                description=ds_agent.description,
                mcp_ids=[ds_mcp.id],
                llm_id=ds_agent.llm_id)

In [ ]:
response = ds_agent.generate("describe the content of table accounts in schema_1")
print(f"Response: {response}")